In [3]:
import pandas as pd
import numpy as np
import re

AQ_PATH = "/Users/liyanwang/Desktop/2026winterProject/stat946/Air-Quality-Data-main/data/NeatData/AirQuality_ON_2022_2024.csv"
WX_PATH = "/Users/liyanwang/Desktop/2026winterProject/stat946/Air-Quality-Data-main/data/NeatData/weather_ON_2022_2024.csv"
TR_PATH = "/Users/liyanwang/Desktop/2026winterProject/stat946/Air-Quality-Data-main/data/NeatData/traffic_ON_2022_2024.csv"

OUT_MERGED = "/Users/liyanwang/Desktop/2026winterProject/stat946/Air-Quality-Data-main/data/NeatData/merged_10km_daily_updated.csv"
OUT_REG_TXT = "/Users/liyanwang/Desktop/2026winterProject/stat946/Air-Quality-Data-main/data/NeatData/regression_results_updated.txt"

START_DATE = "2022-04-26"
END_DATE   = "2024-09-26"

# Radius settings within 10km radius
R_MIN_KM = 0
R_MAX_KM = 10

# Helper function
def to_daily(x):
    return pd.to_datetime(x, errors="coerce").dt.normalize()

def haversine_km(lat1, lon1, lat2, lon2):
    """
    Vectorized haversine distance (km).
    lat1, lon1: (N,) arrays
    lat2, lon2: (M,) arrays
    return: (N, M) distance matrix in km
    """
    R = 6371.0
    lat1 = np.deg2rad(lat1)[:, None]
    lon1 = np.deg2rad(lon1)[:, None]
    lat2 = np.deg2rad(lat2)[None, :]
    lon2 = np.deg2rad(lon2)[None, :]
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    c = 2*np.arcsin(np.sqrt(a))
    return R * c

def traffic_wide_to_long(tr_df):
    """
    traffic is wide: xYYYY_MM_DD columns -> long (camera_id, date, traffic)
    """
    date_cols = [c for c in tr_df.columns if re.fullmatch(r"x\d{4}_\d{2}_\d{2}", str(c))]
    id_cols = [c for c in tr_df.columns if c not in date_cols]
    long_df = tr_df.melt(
        id_vars=id_cols, value_vars=date_cols,
        var_name="date_col", value_name="traffic"
    )
    long_df["date"] = pd.to_datetime(
        long_df["date_col"].str[1:], format="%Y_%m_%d", errors="coerce"
    ).dt.normalize()
    long_df["traffic"] = pd.to_numeric(long_df["traffic"], errors="coerce")
    return long_df.drop(columns=["date_col"])

In [4]:
# Read data
aq = pd.read_csv(AQ_PATH, low_memory=False)
wx = pd.read_csv(WX_PATH, low_memory=False)
tr = pd.read_csv(TR_PATH, low_memory=False)

# Standardize AQ (base table)
# Date, Lat/Lon
if "Date" in aq.columns:
    aq["date"] = to_daily(aq["Date"])
elif "date" in aq.columns:
    aq["date"] = to_daily(aq["date"])
else:
    raise ValueError("AQ: cannot find date column ('Date' or 'date').")
AQ_LAT, AQ_LON = "Latitude", "Longitude"
for c in [AQ_LAT, AQ_LON, "Station ID"]:
    if c not in aq.columns:
        raise ValueError(f"AQ: missing column {c}")
# Filter date range
aq = aq[aq["date"].between(START_DATE, END_DATE)].copy()
# Unique AQ stations for spatial matching
aq_stations = (
    aq[["Station ID", AQ_LAT, AQ_LON]]
    .dropna()
    .drop_duplicates()
    .reset_index(drop=True)
)
aq_stations["aq_sid"] = aq_stations["Station ID"].astype(str)

# Weather: daily per weather station
# Check weather date, lat/lon, temperature and precip
if "Date/Time" in wx.columns:
    wx["date"] = to_daily(wx["Date/Time"])
elif "date" in wx.columns:
    wx["date"] = to_daily(wx["date"])
else:
    raise ValueError("Weather: cannot find date column ('Date/Time' or 'date').")

WX_LAT, WX_LON = "Latitude (y)", '"Longitude (x)"'
for c in [WX_LAT, WX_LON]:
    if c not in wx.columns:
        raise ValueError(f"Weather: missing column {c}")

# Mean air temperature – affects chemical reaction rates
WX_MEAN_TEMP_COL = "Mean Temp (°C)"
# Max & Min temp used to compute diurnal temperature range, reflecting atmospheric stability and vertical dispersion
WX_MAX_TEMP_COL  = "Max Temp (°C)"
WX_MIN_TEMP_COL  = "Min Temp (°C)"
# Total precip captures wet deposition that removes pollutants from air
WX_TOTAL_PRECIP_COL = "Total Precip (mm)"
# Total rainfall directly linked to pollutant washout than total precip (rain + snow)
WX_TOTAL_RAIN_COL   = "Total Rain (mm)"
# Snow on ground indicates persistent winter surface conditions, especially useful in Canada winter
WX_SNOW_GROUND_COL  = "Snow on Grnd (cm)"
# Maximum gust speed – proxy for atmospheric ventilation and pollutant dispersion
WX_MAX_GUST_SPD_COL = "Spd of Max Gust (km/h)"
WX_COLS = [
    WX_MEAN_TEMP_COL,
    WX_MAX_TEMP_COL,
    WX_MIN_TEMP_COL,
    WX_TOTAL_PRECIP_COL,
    WX_TOTAL_RAIN_COL,
    WX_SNOW_GROUND_COL,
    WX_MAX_GUST_SPD_COL,
]
missing = [c for c in WX_COLS if c not in wx.columns]
if missing:
    raise ValueError(f"Weather: missing columns: {missing}")
# Filter date range
wx = wx[wx["date"].between(START_DATE, END_DATE)].copy()
# Unique Weather stations for spatial matching
wx_sites = (
    wx[["Station ID", WX_LAT, WX_LON]]
    .dropna()
    .drop_duplicates()
    .reset_index(drop=True)
)
wx_sites["wx_sid"] = wx_sites["Station ID"].astype(str)
# Keep daily weather table keyed by wx_sid + date with all predictors
wx_daily = (
    wx.merge(wx_sites[["Station ID", "wx_sid"]], on="Station ID", how="left")[
        ["wx_sid", "date"] + WX_COLS
    ].copy()
)
for c in WX_COLS:
    wx_daily[c] = pd.to_numeric(wx_daily[c], errors="coerce")

# Traffic: wide -> long (camera daily)
# Convert wide to long
if any(re.fullmatch(r"x\d{4}_\d{2}_\d{2}", str(c)) for c in tr.columns):
    tr_long = traffic_wide_to_long(tr)
else:
    tr_long = tr.copy()
    if "date" in tr_long.columns:
        tr_long["date"] = to_daily(tr_long["date"])
    else:
        raise ValueError("Traffic: cannot find xYYYY_MM_DD columns or 'date' column.")
# Filter date range
tr_long = tr_long[tr_long["date"].between(START_DATE, END_DATE)].copy()
TR_LAT, TR_LON = "latitude", "longitude"
for c in [TR_LAT, TR_LON]:
    if c not in tr_long.columns:
        raise ValueError(f"Traffic: missing column {c}")

# define a traffic site id (camera id if exists; else lat/lon)
if "traffic_camera" in tr_long.columns:
    tr_long["tr_sid"] = tr_long["traffic_camera"].astype(str)
else:
    tr_long["tr_sid"] = (tr_long[TR_LAT].astype(str) + "_" + tr_long[TR_LON].astype(str))
tr_sites = (
    tr_long[["tr_sid", TR_LAT, TR_LON]]
    .dropna()
    .drop_duplicates()
    .reset_index(drop=True)
)
tr_daily = tr_long[["tr_sid", "date", "traffic"]].copy()


# 7) Build spatial pairs (AQ station -> nearby WX/TR sites)
# AQ -> Weather pairs
aq_lat = pd.to_numeric(aq_stations[AQ_LAT], errors="coerce").to_numpy()
aq_lon = pd.to_numeric(aq_stations[AQ_LON], errors="coerce").to_numpy()
wx_lat = pd.to_numeric(wx_sites[WX_LAT], errors="coerce").to_numpy()
wx_lon = pd.to_numeric(wx_sites[WX_LON], errors="coerce").to_numpy()
d_aw = haversine_km(aq_lat, aq_lon, wx_lat, wx_lon)
mask_aw = (d_aw >= R_MIN_KM) & (d_aw <= R_MAX_KM)
i, j = np.where(mask_aw)
aq_wx_pairs = pd.DataFrame({
    "aq_sid": aq_stations.loc[i, "aq_sid"].to_numpy(),
    "wx_sid": wx_sites.loc[j, "wx_sid"].to_numpy()
}).drop_duplicates()

# AQ -> Traffic pairs
tr_lat = pd.to_numeric(tr_sites[TR_LAT], errors="coerce").to_numpy()
tr_lon = pd.to_numeric(tr_sites[TR_LON], errors="coerce").to_numpy()
d_at = haversine_km(aq_lat, aq_lon, tr_lat, tr_lon)
mask_at = (d_at >= R_MIN_KM) & (d_at <= R_MAX_KM)
i2, j2 = np.where(mask_at)
aq_tr_pairs = pd.DataFrame({
    "aq_sid": aq_stations.loc[i2, "aq_sid"].to_numpy(),
    "tr_sid": tr_sites.loc[j2, "tr_sid"].to_numpy()
}).drop_duplicates()

# Aggregate WX/TR to AQ station-day
# Weather aggregation to AQ station-day
aq_wx_daily = (
    aq_wx_pairs.merge(wx_daily, on="wx_sid", how="left")
              .groupby(["aq_sid", "date"], as_index=False)
              .agg(
                  MeanTemp=(WX_MEAN_TEMP_COL, "mean"),
                  TotalPrecip=(WX_TOTAL_PRECIP_COL, "mean"),
                  MaxTemp=(WX_MAX_TEMP_COL, "mean"),
                  MinTemp=(WX_MIN_TEMP_COL, "mean"),
                  TotalRain=(WX_TOTAL_RAIN_COL, "mean"),
                  SnowOnGround=(WX_SNOW_GROUND_COL, "median"),
                  MaxGustSpd=(WX_MAX_GUST_SPD_COL, "mean"),
                  wx_n=("wx_sid", "nunique")
              )
)
aq_wx_daily["TempRange"] = aq_wx_daily["MaxTemp"] - aq_wx_daily["MinTemp"]

# Traffic aggregation to AQ station-day
aq_tr_daily = (
    aq_tr_pairs.merge(tr_daily, on="tr_sid", how="left")
              .groupby(["aq_sid", "date"], as_index=False)
              .agg(
                  TotalTraffic=("traffic", "sum"),
                  tr_n=("tr_sid", "nunique")  # number of unique cameras used
              )
)

# 9) Merge back to AQ base
aq_base = aq.copy()
aq_base["aq_sid"] = aq_base["Station ID"].astype(str)
merged = (
    aq_base.merge(aq_wx_daily, on=["aq_sid", "date"], how="left")
           .merge(aq_tr_daily, on=["aq_sid", "date"], how="left")
)
merged = merged.drop(columns=["aq_sid"])

# Save
merged.to_csv(OUT_MERGED, index=False)

print(f"Radius rule: {R_MIN_KM}–{R_MAX_KM} km")
print("AQ rows:", aq.shape[0], "Merged rows:", merged.shape[0])
print("Saved:", OUT_MERGED)
print("Merged columns:", merged.columns.tolist())


Radius rule: 0–10 km
AQ rows: 33535 Merged rows: 33535
Saved: /Users/liyanwang/Desktop/2026winterProject/stat946/Air-Quality-Data-main/data/NeatData/merged_10km_daily_updated.csv
Merged columns: ['Date', 'Station ID', 'Latitude', 'Longitude', 'CO_subAQI', 'NO2_subAQI', 'O3_subAQI', 'PM2.5_subAQI', 'SO2_subAQI', 'AQI', 'date', 'MeanTemp', 'TotalPrecip', 'MaxTemp', 'MinTemp', 'TotalRain', 'SnowOnGround', 'MaxGustSpd', 'wx_n', 'TempRange', 'TotalTraffic', 'tr_n']
